In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

import cv2

In [2]:
from sklearn.model_selection import train_test_split

import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class createDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features)
        self.labels =  torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

class NeuralNetwork(nn.Module):	
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 3 input image channel, 6 output channels, 
        # 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        # max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)# 34x34 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
if __name__ == '__main__':
    
    photos_path = './photos/'

    # lists to hold each data value and their respective label
    features, labels = [], []

    encoding = {}
    # re-sizing images and extracting RGB
    for i, dir in enumerate(os.listdir(photos_path)):
        encoding[dir] = i
        for file in os.listdir(os.path.join(photos_path, dir)):
            img = cv2.imread(os.path.join(photos_path, dir, file))
            img = cv2.resize(img, dsize=(32,32))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            img = (img / 127.5) - 1.0
                        
#             plt.imshow(img)
#             plt.show()
                        
            features.append(img.T)
            labels.append(i)
            

    # split data 
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 1/4)

    
    # create dataset object using torch parent class 'Dataset'
    train_data = createDataset(np.array(X_train), np.array(y_train))
    test_data = createDataset(np.array(X_test), np.array(y_test))

    # creating DataLoaders which feed batch_size number of images to the NN 
    batch_size = 5
    train_loader = DataLoader(train_data, batch_size)
    test_loader = DataLoader(test_data, batch_size)

    model = NeuralNetwork()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=.001, momentum=0.9)

    model = model.float()


    for epoch in range(100):

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):

            inputs, labels = data
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 9 == 0:    # print every 2 mini-batches
                print('Epoch')
                print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
                running_loss = 0.0

    
    # prepare to count predictions for each class
    classes = os.listdir('./photos')
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # no gradients required
    with torch.no_grad():
        for data in test_loader:
            features, labels = data
            outputs = model(features.float())
            _, predictions = torch.max(outputs,1)
            print(outputs, predictions)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Epoch
[1,     1] loss: 0.070
Epoch
[1,    10] loss: 0.620
Epoch
[1,    19] loss: 0.618
Epoch
[1,    28] loss: 0.630
Epoch
[1,    37] loss: 0.628
Epoch
[1,    46] loss: 0.626
Epoch
[2,     1] loss: 0.070
Epoch
[2,    10] loss: 0.617
Epoch
[2,    19] loss: 0.615
Epoch
[2,    28] loss: 0.626
Epoch
[2,    37] loss: 0.625
Epoch
[2,    46] loss: 0.622
Epoch
[3,     1] loss: 0.070
Epoch
[3,    10] loss: 0.613
Epoch
[3,    19] loss: 0.610
Epoch
[3,    28] loss: 0.621
Epoch
[3,    37] loss: 0.620
Epoch
[3,    46] loss: 0.616
Epoch
[4,     1] loss: 0.070
Epoch
[4,    10] loss: 0.605
Epoch
[4,    19] loss: 0.601
Epoch
[4,    28] loss: 0.609
Epoch
[4,    37] loss: 0.610
Epoch
[4,    46] loss: 0.601
Epoch
[5,     1] loss: 0.069
Epoch
[5,    10] loss: 0.586
Epoch
[5,    19] loss: 0.576
Epoch
[5,    28] loss: 0.581
Epoch
[5,    37] loss: 0.587
Epoch
[5,    46] loss: 0.560
Epoch
[6,     1] loss: 0.067
Epoch
[6,    10] loss: 0.534
Epoch
[6,    19] loss: 0.504
Epoch
[6,    28] loss: 0.502
Epoch
[6,    3

Epoch
[49,    19] loss: 0.016
Epoch
[49,    28] loss: 0.002
Epoch
[49,    37] loss: 0.027
Epoch
[49,    46] loss: 0.030
Epoch
[50,     1] loss: 0.001
Epoch
[50,    10] loss: 0.054
Epoch
[50,    19] loss: 0.009
Epoch
[50,    28] loss: 0.015
Epoch
[50,    37] loss: 0.019
Epoch
[50,    46] loss: 0.011
Epoch
[51,     1] loss: 0.001
Epoch
[51,    10] loss: 0.025
Epoch
[51,    19] loss: 0.055
Epoch
[51,    28] loss: 0.052
Epoch
[51,    37] loss: 0.023
Epoch
[51,    46] loss: 0.003
Epoch
[52,     1] loss: 0.000
Epoch
[52,    10] loss: 0.014
Epoch
[52,    19] loss: 0.024
Epoch
[52,    28] loss: 0.002
Epoch
[52,    37] loss: 0.005
Epoch
[52,    46] loss: 0.081
Epoch
[53,     1] loss: 0.000
Epoch
[53,    10] loss: 0.020
Epoch
[53,    19] loss: 0.053
Epoch
[53,    28] loss: 0.005
Epoch
[53,    37] loss: 0.009
Epoch
[53,    46] loss: 0.068
Epoch
[54,     1] loss: 0.000
Epoch
[54,    10] loss: 0.016
Epoch
[54,    19] loss: 0.031
Epoch
[54,    28] loss: 0.001
Epoch
[54,    37] loss: 0.004
Epoch
[54,

Epoch
[96,    46] loss: 0.001
Epoch
[97,     1] loss: 0.000
Epoch
[97,    10] loss: 0.000
Epoch
[97,    19] loss: 0.001
Epoch
[97,    28] loss: 0.000
Epoch
[97,    37] loss: 0.000
Epoch
[97,    46] loss: 0.001
Epoch
[98,     1] loss: 0.000
Epoch
[98,    10] loss: 0.000
Epoch
[98,    19] loss: 0.001
Epoch
[98,    28] loss: 0.000
Epoch
[98,    37] loss: 0.000
Epoch
[98,    46] loss: 0.001
Epoch
[99,     1] loss: 0.000
Epoch
[99,    10] loss: 0.000
Epoch
[99,    19] loss: 0.001
Epoch
[99,    28] loss: 0.000
Epoch
[99,    37] loss: 0.000
Epoch
[99,    46] loss: 0.001
Epoch
[100,     1] loss: 0.000
Epoch
[100,    10] loss: 0.000
Epoch
[100,    19] loss: 0.001
Epoch
[100,    28] loss: 0.000
Epoch
[100,    37] loss: 0.000
Epoch
[100,    46] loss: 0.000
tensor([[-4.1397,  3.6388],
        [ 6.0846, -6.2447],
        [ 7.9544, -8.5262],
        [-8.6579,  7.8066],
        [-7.9917,  7.2365]]) tensor([1, 0, 0, 1, 1])
tensor([[ 12.6182, -13.0878],
        [ -0.2193,  -0.0675],
        [ -9.3659, 